In [2]:
import os
import random
import sys
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

# Использовать только процессор.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# Изменить уровень отображения логов
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

# Корень проекта.
DIR_ROOT = Path.cwd().parent.parent
sys.path.append(str(DIR_ROOT))
import src.model_utils as mu
from src.logger import mean_callback_epoch, log_to_json, log_to_table

# Путь к удаленной директории с ресурсами: данные, модели и т.д.
DIR_REMOTE: Path | None = Path('/home/admin/cafa/resources')

if DIR_REMOTE is not None and DIR_REMOTE.exists():
    DIR_RESOURCE = DIR_REMOTE
else:
    DIR_RESOURCE = DIR_ROOT

# Путь к директории с подготовленными данными.
DIR_PREPARED_DATA = DIR_RESOURCE / 'data/prepared'
# Путь к файлу с оценкой моделей для различных экспериментов.
PATH_TO_EVAL_FILE = DIR_ROOT / 'experiments/eval.json'

# Утилиты

In [2]:
def print_shape_and_total_memory(df: pd.DataFrame, prefix: str = '') -> None:
    '''
    Вывод информации о форме и общем размере занятой памяти
    объектаpd.DataFrame
    '''
    print(
        prefix,
        f'Shape: {df.shape}',
        f'Total memory: {df.memory_usage().sum() / 1024**3:.1f} GB',
        '---',
        sep='\n',
    )


# Загрузка данных (признаков и целей)

In [3]:
PATH_TO_FEATURES = DIR_PREPARED_DATA / 'train_features.csv'
PATH_TO_LABELS = DIR_PREPARED_DATA / 'train_lbls_top1500_goterms.csv'
# Загрузка признаков.
x = pd.read_csv(PATH_TO_FEATURES)
# Загрузка целей.
y = pd.read_csv(PATH_TO_LABELS)
y = y.astype('int32')
print_shape_and_total_memory(x, 'x:')
print_shape_and_total_memory(y, 'y:')

x:
Shape: (142246, 1024)
Total memory: 1.1 GB
---
y:
Shape: (142246, 1500)
Total memory: 0.8 GB
---


# Оценка модели

## Оценка производительности модели
Оценка производительности модели через KFold кросс-валидацию при сочетании  
параметров: 

Структура слоев:  
- [512, 512, 512]  
- [1024, 512, 1024]  
- [512, 256, 512]  

Набор dropout:   
- {0.1, 0.3, 0.5}

Повторяем эксперименты 15 и 17 с параметрами EarlyStopping   
('min_delta': 0.005, 'patience': 20`)  
15	dt2023_10Aug22_29_10 None [1024, 512, 1024]	EarlyStopping	0.1	0.41 0.01  
17	dt2023_11Aug00_25_08 None [1024, 512, 1024]	EarlyStopping	0.5	0.40 0.00

In [4]:
ACTIVATION = 'relu'
BATCH_SIZE = 5000
# Список для создания `Сallbacks` объектов. 
CALLBACKS_PARAMS: list[mu.CallbackParams] = [
    {
        'callback': tf.keras.callbacks.EarlyStopping,
        'params': {
            'monitor': 'val_loss',
            'mode': 'min',
            'min_delta': 0.01,
            'patience': 10,
            'restore_best_weights': True,
        }
    },
]
# None или результат `mu.create_callbacks(CALLBACKS_PARAMS)`
CALLBACKS = mu.create_callbacks(CALLBACKS_PARAMS)
DROPOUT = 0.1
EPOCHS = 150
KERNEL_REGULARIZER = None
# Структура скрытых слоев в последовательной DNN.
LAYERS_STRCT = [512, 512, 512]
LEARNING_RATE = 0.001
# Число циклов с кросс-валидацией.
N_REPEATS = 1
# Число фолдов в кросс-валидации.
N_SPLITS = 5
# Список метрик, используемых для рассчета производительности модели.
METRICS = [mu.f1_score_micro,]
SHUFFLE = True
VALIDATION_SPLIT = 0.1
VERBOSE = 1


# Формирование фабрики скомпилированных моделей.
mfabric = mu.ModelCompileFabric(
    activation=ACTIVATION,
    kernel_regularizer=KERNEL_REGULARIZER,
    dropout=DROPOUT,
    layers_strct=LAYERS_STRCT,
    learning_rate=LEARNING_RATE,
)
# Создаем прокси для обучения моделей.
mproxy = mu.ProxyFitModel(
    mfabric,
    batch_size=BATCH_SIZE,
    callbacks=CALLBACKS,
    epochs=EPOCHS,
    shuffle=SHUFFLE,
    validation_split=VALIDATION_SPLIT,
    verbose=VERBOSE,
)
random_state = random.randint(0, 10_000)
# Оценка производительности модели через кросс-валидацию.
results, all_epoch_history = mu.evaluate_model(
    features=x.to_numpy(),
    lbls=y.to_numpy(),
    metrics=METRICS,  # type: ignore
    proxy_model=mproxy,
    n_repeats=N_REPEATS,
    n_splits=N_SPLITS,
    random_state=random_state,
)

Epoch 1/150
21/21 [==============================] - 11s 429ms/step - loss: 0.2348 - binary_accuracy: 0.9258 - val_loss: 0.5649 - val_binary_accuracy: 0.9847
Epoch 2/150
21/21 [==============================] - 9s 408ms/step - loss: 0.0852 - binary_accuracy: 0.9792 - val_loss: 0.5558 - val_binary_accuracy: 0.9846
Epoch 3/150
21/21 [==============================] - 9s 415ms/step - loss: 0.0787 - binary_accuracy: 0.9795 - val_loss: 0.5250 - val_binary_accuracy: 0.9845
Epoch 4/150
21/21 [==============================] - 9s 411ms/step - loss: 0.0759 - binary_accuracy: 0.9797 - val_loss: 0.5078 - val_binary_accuracy: 0.9845
Epoch 5/150
21/21 [==============================] - 9s 409ms/step - loss: 0.0733 - binary_accuracy: 0.9798 - val_loss: 0.4880 - val_binary_accuracy: 0.9847
Epoch 6/150
21/21 [==============================] - 9s 411ms/step - loss: 0.0710 - binary_accuracy: 0.9799 - val_loss: 0.4645 - val_binary_accuracy: 0.9848
Epoch 7/150
21/21 [==============================] - 9s 4

## Сохранение результатов
Сохранение результатов оценки производительности модели и сопутсвующих параметров. 

In [14]:
data_info_dict = {
    'file_name_features': PATH_TO_FEATURES.name,
    'file_name_labels': PATH_TO_LABELS.name,
}
model_params_dict = {
    'activation': ACTIVATION,
    'batch_size': BATCH_SIZE,
    'callbacks': (
        CALLBACKS if CALLBACKS is None
        else mu.create_callbacks_info(CALLBACKS_PARAMS)
    ),
    'dropout': DROPOUT,
    'epochs': EPOCHS,
    'kernel_regularizer': KERNEL_REGULARIZER,
    'layers_strct': LAYERS_STRCT,
    'learning_rate': LEARNING_RATE,
    'n_repeats': N_REPEATS,
    'n_splits': N_SPLITS,
    'random_state': random_state,
    'shuffle': SHUFFLE,
    'validation_split': VALIDATION_SPLIT,
    'mean_callback_epoch': mean_callback_epoch(all_epoch_history)
}
# Формируем словарь с текущей оценкой производительности модели
# и сопутсвующими параметрами.
model_eval_params: mu.ModelEvalParams = {
    'data_info': data_info_dict,
    'model_params': model_params_dict,
    'scores': mu.get_scores_stats(results),
}
cur_datetime = datetime.now().strftime('%Y_%d%b%H_%M_%S')
# Записываем результаты текущей оценки производительности модели в файл.
log_to_json(
    log_file=DIR_ROOT / 'experiments/eval.json',
    params={f'dt{cur_datetime}': model_eval_params},
)

Файл был дополнен


In [3]:
log_to_table(DIR_ROOT / 'experiments/eval.json')

,model_name,kernel_regularizer,layers_strct,callback,dropout,mean_callback_epoch,mean_f1_score_micro,std_f1_score_micro
0,dt2023_10Aug00_51_32,None,"[512, 512, 512]",None,0.0,NaN,0.33,0.01
1,dt2023_10Aug01_02_16,None,"[512, 512, 512]",None,0.0,NaN,0.35,0.01
2,dt2023_10Aug01_27_45,None,"[512, 512, 512]",EarlyStopping,0.0,NaN,0.36,0.01
3,dt2023_10Aug02_05_30,None,"[512, 512, 512]",EarlyStopping,0.0,NaN,0.39,0.01
4,dt2023_10Aug02_48_17,None,"[512, 512, 512]",None,0.0,NaN,0.40,0.01
5,dt2023_10Aug11_23_28,None,"[512, 512, 512]",None,0.0,NaN,0.42,0.02
6,dt2023_10Aug15_30_51,l1,"[512, 512, 512]",None,0.0,NaN,0.17,0.00
7,dt2023_10Aug16_01_52,l1,"[512, 512, 512]",EarlyStopping,0.1,NaN,0.16,0.01
8,dt2023_10Aug16_56_37,l1,"[512, 512, 512]",EarlyStopping,0.0,NaN,0.16,0.01
9,dt2023_10Aug17_30_38,None,"[512, 512, 512]",EarlyStopping,0.0,NaN,0.38,0.02


### Выводы

1. Наилучший результат показала модель №15 с [1024, 512, 1024] и `dropuot` 0.1, но  
   превосходит другие незначительно, возможно погрешность.

2. Применение регуляризации сильно снижает показатели ~ 0.15-0.17  
   по mean_f1_score_micro

3. Регулировка параметров EarlyStopping к значительным различиям в качестве  
   моделей не привела (см. 15 запуск и 21, 17 запуск и 22 запуск.)